In [43]:
import pandas as pd
import numpy as np
import datetime as dt
from w3lib.html import remove_tags
import re

In [120]:
def longToShortDate(x, sep):
    months = ['janeiro', 'fevereiro','março', 'abril', 'maio', 'junho', 'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro']
    months_dic = {value:idx+1 for idx, value in enumerate(months)}
    date = [i.strip() for i in x.split(sep)]
    return f'{date[0]}/{months_dic[date[1]]}/{date[2]}'

# convert to datetime object
def convertToDatetime(x, function=longToShortDate):
    x = x.lower().replace(',','')
    x = dt.datetime.strptime(function(x, '/'), "%d/%m/%Y")
    return x

last_date = convertToDatetime('25 / Outubro / 2021')
today = dt.datetime.today()
delta = today - last_date
delta.days

5

In [44]:
import os
while not os.getcwd().endswith("Data Cleaning"):
    os.chdir("..")

os.getcwd()

'c:\\Users\\gilnr\\OneDrive - NOVASBE\\Work Project\\Thesis - Code\\Data Cleaning'

In [45]:
from pathlib import Path, PureWindowsPath

main_folder = PureWindowsPath("c:\\Users\\gilnr\\OneDrive - NOVASBE\\Work Project\\Thesis - Code")
MAIN_FOLDER = Path(main_folder)
DATA_FOLDER = MAIN_FOLDER / "Data"
DATA_FOLDER

WindowsPath('c:/Users/gilnr/OneDrive - NOVASBE/Work Project/Thesis - Code/Data')

# Load all datasets 

In [46]:
bons_empregos = pd.read_json(DATA_FOLDER / 'bons_empregos_jobs.json')
bons_empregos_2 = pd.read_json(DATA_FOLDER / 'BonsEmpregos.json', lines=True)
career_jet = pd.read_json(DATA_FOLDER / 'career_jet_api.json')
carga_de_trabalhos = pd.read_json(DATA_FOLDER / 'CargaDeTrabalhos.json', lines=True)
emprego_xl_2 = pd.read_json(DATA_FOLDER / 'emprego_xl_jobs.json')
emprego_xl = pd.read_json(DATA_FOLDER / 'EmpregoXl.json', lines=True)
emprego_org = pd.read_json(DATA_FOLDER / 'EmpregoOrg.json')
itjobs = pd.read_json(DATA_FOLDER / 'itjobs_api.json')
jooble = pd.read_json(DATA_FOLDER / 'jooble_api.json')
landing_jobs = pd.read_json(DATA_FOLDER / 'landingjobs_api.json')
net_empregos_2 = pd.read_json(DATA_FOLDER / 'net_empregos.json')
net_empregos = pd.read_json(DATA_FOLDER / 'NetEmpregos.json', lines=True)

In [47]:
dataframes = [bons_empregos, bons_empregos_2, career_jet, carga_de_trabalhos, emprego_xl, 
              emprego_xl_2, emprego_org, itjobs, jooble, landing_jobs, net_empregos, net_empregos_2]
total = 0
for i in dataframes:
    print(f'Number of job vacancies: {len(i)}')
    total += len(i)
print(f'Total job vacancies before processing {total}')

Number of job vacancies: 2576
Number of job vacancies: 2427
Number of job vacancies: 6534
Number of job vacancies: 240
Number of job vacancies: 39460
Number of job vacancies: 36318
Number of job vacancies: 99
Number of job vacancies: 3890
Number of job vacancies: 1093
Number of job vacancies: 717
Number of job vacancies: 9012
Number of job vacancies: 54800
Total job vacancies before processing 157166


In [48]:
jobs_dfs = [bons_empregos, career_jet, carga_de_trabalhos, emprego_xl, emprego_org, itjobs, jooble, landing_jobs, net_empregos]
websites = ['Bons empregos', 'Career Jet', 'Carga de Trabalhos', 'Emprego XL', 'Emprego.org','ITjobs','Jooble','Landing Jobs','Net-empregos']
total = 0
for idx,i in enumerate(jobs_dfs):
    print(f'Number of job vacancies: {len(i)}, {websites[idx]}')
    total += len(i)
print(f'Total job vacancies before processing {total}')

Number of job vacancies: 2576, Bons empregos
Number of job vacancies: 6534, Career Jet
Number of job vacancies: 240, Carga de Trabalhos
Number of job vacancies: 39460, Emprego XL
Number of job vacancies: 99, Emprego.org
Number of job vacancies: 3890, ITjobs
Number of job vacancies: 1093, Jooble
Number of job vacancies: 717, Landing Jobs
Number of job vacancies: 9012, Net-empregos
Total job vacancies before processing 63621


In [49]:
a = pd.DataFrame({'0':[np.nan, 1, np.nan, np.nan, np.nan, 2, np.nan, 3, 4, 5, np.nan, np.nan, np.nan]})
a['0'].ffill(limit=1).bfill()

0     1.0
1     1.0
2     1.0
3     2.0
4     2.0
5     2.0
6     2.0
7     3.0
8     4.0
9     5.0
10    5.0
11    NaN
12    NaN
Name: 0, dtype: float64

# Data Cleaning
- For each website there are specific categories that we need to attend. Some require filtering for job location, others cleaning the job description, and the post date.

## Drop Job Vacancies
What makes a unique job vacancy?
- For our analysis it will be: [job_title, job_description, company, job_location]

# General Functions

In [92]:
def copy_df(dataframe):
   return dataframe.copy()

def replacenan(dataframe):
    dataframe.replace('nan', np.nan, inplace=True)
    return dataframe
    
def dropNullJobs(dataframe):
    """
    Drop null values that make an online job vacancy unusable for analysis.
    The subset to drop is: ['post_date', 'job_title', 'job_description']
    """
    dataframe.dropna(subset=['post_date', 'job_title', 'job_description'], inplace=True)
    return dataframe

# remove duplicates
def removeDupes(dataframe, subset=['job_title', 'job_description', 'company', 'job_location']):
    dataframe.sort_values(by='post_date').drop_duplicates(subset=subset, keep='last', inplace=True)
    return dataframe

def listToRows(dataframe, column):
    return dataframe.explode(column)

def removeTags(dataframe, column_list):
    for i in column_list:
        dataframe[i] = dataframe[i].apply(remove_tags)
    return dataframe

# Description
def clean_text(text):
    to_replace = ['\r', '\n', '•']
    replace = [' ', ' ', '\n']

    for idx, val in enumerate(to_replace):
        text = text.replace(val, replace[idx])
    text = text.strip()
    return text

def cleanDescription(dataframe, column_list):
    for i in column_list:
        dataframe[i] = dataframe[i].apply(lambda x: clean_text(x))
    return dataframe

def invertDate(x):
    if type(x) == float:
        return np.nan
    date = x.split('-')
    return date[2].strip()+'-'+date[1].strip()+'-'+date[0].strip()

def pipeInvertDate(dataframe, function=invertDate):
    dataframe['post_date'] = dataframe['post_date'].apply(lambda x: function(x))
    return dataframe

# Date Related Functions

In [51]:
def postDatePreprocess(dataframe, sep=" "):
    dataframe['post_date'] = dataframe['post_date'].apply(lambda x: x.split(sep)[0]) 
    return dataframe

def postDateFillNa(dataframe):
    dataframe['post_date'] = dataframe['post_date'].ffill(limit=1).bfill()
    return dataframe

# convert portuguese months to numbers
def longToShortDate(x, sep):
    months = ['janeiro', 'fevereiro','março', 'abril', 'maio', 'junho', 'julho', 'agosto', 'setembro', 'outubro', 'novembro', 'dezembro']
    months_dic = {value:idx+1 for idx, value in enumerate(months)}
    date = [i.strip() for i in x.split(sep)]
    return f'{date[0]}/{months_dic[date[1]]}/{date[2]}'

# convert to datetime object
def convertToDatetime(dataframe, function, sep=' '):
    # Remove comma from date
    dataframe['post_date'] = dataframe['post_date'].apply(lambda x: str(x).lower().replace(',',''))
    dataframe['post_date'] = dataframe['post_date'].apply(lambda x: dt.datetime.strptime(function(x, sep), "%d/%m/%Y"))
    return dataframe

# Convert Scrape date to datetime
def toDatetime(dataFrame, columns_list):
    for i in columns_list:
        dataFrame[i] = pd.to_datetime(dataFrame[i])
    return dataFrame
    
def notDateToNan(x):
    if re.findall('(0[1-9]|[12][0-9]|3[01])[-](0[1-9]|1[012])[-](19|20)\d\d', str(x)) != []:
        return x
    else:
        return np.nan

def applyFuncToColumn(dataframe, function=notDateToNan, columns_list=['post_date']):
    for i in columns_list:
        dataframe[i] = dataframe[i].apply(lambda x: function(x))
    return dataframe

## Define `Clean Job Title` Functions

In [52]:
import functools
from typing import Callable

ComposableFunction = Callable[[str], str]

def compose(*functions: ComposableFunction) -> ComposableFunction:
    return functools.reduce(lambda f, g: lambda x: g(f(x)), functions)

def cleanJobChars(x: str) -> str:
    # Capitalize the job title
    x = x.lower()
    stop_chars = ['(m/f)', 'm/f', '-', ' - ', ' – ', '(remote)', ' / ', '(', 'para']
    ex = False
    for i in stop_chars:
        if i in x:
            if i == '-':
                for exception in ['-se', '-o', '-a', '-os', '-as', 'e-', '-e']:
                    if exception in x:
                        ex = True
                if ex == False:
                    x = x.split(i)[0]
                pass
            else:
                x = x.split(i)[0]     
    return x.strip()

def replaceGenderWords(x: str) -> str:
    gender_words = ['/a', '/o', '/as', '/os', '/e', '/es']
    for i in gender_words:
        x = x.replace(i, '')
    return x

def replaceCommonFillers(x: str) -> str:
    fillers = ['recruta-se', 'oferta de emprego:', 'oferta:', 'oferta de emprego', 'oferta', 'precisa-se', 'precisas-se']
    for i in fillers:
        x = x.replace(i, '')
    return x

cleanJobTitle = compose(replaceCommonFillers, replaceGenderWords, cleanJobChars)

# How to use:
# applyFuncToColumn(bons_empregos, function=cleanJobTitle, columns_list=['job_title'])

# Total jobs grouped by Year and Month

In [53]:
def totalJobsByYearMonth(dataframe):
    dataframe['post_year'] = dataframe['post_date'].dt.year
    dataframe['post_month'] = dataframe['post_date'].dt.month
    return pd.DataFrame(dataframe.groupby(['post_year', 'post_month'])['job_title'].count()).sort_values(by=['post_year', 'post_month'], ascending=False)

## Bons Empregos
- Specific functions:
    - `getPortugalLocation`

In [54]:
def getPortugalLocation(dataframe):
    # Get only job offers in Portugal
    dataframe = dataframe.loc[dataframe['job_location'] != 'Estrangeiro'].copy()
    return dataframe

In [55]:
bons_empregos_clean = (bons_empregos.
                    pipe(copy_df).
                    pipe(replacenan).
                    pipe(postDateFillNa).
                    pipe(dropNullJobs).
                    pipe(applyFuncToColumn, function=cleanJobTitle, columns_list=['job_title']).
                    pipe(toDatetime, ['scrape_date']).
                    pipe(getPortugalLocation).
                    pipe(convertToDatetime, longToShortDate).
                    pipe(removeDupes)
)

print(f'Previous shape: {bons_empregos.shape}\nCurrent shape:{bons_empregos_clean.shape}')
bons_empregos_clean.head()

Previous shape: (2576, 8)
Current shape:(2359, 8)


,job_location,job_category,job_description,job_title,post_date,scrape_date,job_href,company
10,Porto,Informática (Programação),"Na Dellent, ligamos os melhores talentos aos m...",backend developer,2021-10-01,2021-05-10,https://www.bonsempregos.com/oferta-emprego/ba...,Dellent
11,Porto,Informática (Programação),"Génération, une entreprise innovante, motivant...",assistant de gestion assurance,2021-10-01,2021-05-10,https://www.bonsempregos.com/oferta-emprego/as...,GENERATION
12,Porto,Informática (Programação),"Dellent Consulting is a Portuguese consulting,...",business intelligence developer,2021-10-01,2021-05-10,https://www.bonsempregos.com/oferta-emprego/bu...,Dellent
13,Porto,Informática (Programação),A Dellent Consulting é uma consultora especial...,senior python/angular developer,2021-10-01,2021-05-10,https://www.bonsempregos.com/oferta-emprego/se...,Dellent
14,Porto,Informática (Programação),"At Dellent Consulting, we know that with the r...",application maintenance engineer,2021-10-01,2021-05-10,https://www.bonsempregos.com/oferta-emprego/ap...,Dellent


In [56]:
totalJobsByYearMonth(bons_empregos_clean)

job_title
post_year post_month           
2021      10                 10
          9                 227
          8                  82
          7                 244
          6                 246
          5                 245
          4                 248
          3                 229
          2                 174
          1                 159
2020      12                117
          11                174
          10                204

## Career Jet

In [57]:
# convert job location to list
career_jet['job_location'] = career_jet['job_location'].apply(lambda x: x.split(','))

In [58]:
career_jet_clean = (career_jet.
                    pipe(copy_df).
                    pipe(replacenan).
                    pipe(postDateFillNa).
                    pipe(dropNullJobs).
                    pipe(applyFuncToColumn, function=cleanJobTitle, columns_list=['job_title']).
                    pipe(toDatetime, ['scrape_date', 'post_date']).
                    pipe(listToRows, 'job_location').
                    pipe(removeDupes)
)
print(f'Previous shape: {career_jet.shape}\nCurrent shape:{career_jet_clean.shape}')
career_jet_clean.head()

Previous shape: (6534, 9)
Current shape:(10613, 9)


,job_title,job_description,post_date,scrape_date,company,job_location,job_category,job_href,salary
0,empregados mesa,Estamos à procura de funcionários para: - Empr...,2021-10-22,2021-10-22,Restaurante O Caneira,Sintra,,http://jobviewtrack.com/pt-pt/job-1b4941684014...,1 - 1
0,empregados mesa,Estamos à procura de funcionários para: - Empr...,2021-10-22,2021-10-22,Restaurante O Caneira,Lisboa,,http://jobviewtrack.com/pt-pt/job-1b4941684014...,1 - 1
1,community support associate,Who we are Seedrs is the leading equity crow...,2021-10-22,2021-10-22,Seedrs,Lisboa,,http://jobviewtrack.com/pt-pt/job-1b12417e5814...,
2,trabalhador não qualificado da construção de e...,Condições Requeridas Habilitações Escolares ...,2021-10-22,2021-10-22,,Esmoriz,,http://jobviewtrack.com/pt-pt/job-191341795f05...,665 - 665
2,trabalhador não qualificado da construção de e...,Condições Requeridas Habilitações Escolares ...,2021-10-22,2021-10-22,,Aveiro,,http://jobviewtrack.com/pt-pt/job-191341795f05...,665 - 665


In [59]:
assert career_jet_clean.post_date.dtypes == career_jet_clean.scrape_date.dtypes

In [60]:
totalJobsByYearMonth(career_jet_clean)

,,job_title
post_year,post_month,
2021,10,10613


## Carga de Trabalhos

In [62]:
carga_de_trabalhos_clean = (carga_de_trabalhos.
                    pipe(copy_df).
                    pipe(replacenan).
                    pipe(postDateFillNa).
                    pipe(dropNullJobs).
                    pipe(applyFuncToColumn, function=cleanJobTitle, columns_list=['job_title']).
                    pipe(toDatetime, ['scrape_date']).
                    pipe(convertToDatetime, longToShortDate, '/').
                    pipe(removeDupes)
)

print(f'Previous shape: {carga_de_trabalhos.shape}\nCurrent shape:{carga_de_trabalhos_clean.shape}')
carga_de_trabalhos_clean.head()

Previous shape: (240, 7)
Current shape:(240, 7)


,job_description,job_title,post_date,scrape_date,company,job_location,job_href
0,URBAN\nOBRAS\nARQUITETURA E DESIGN\nA URBAN Ob...,procura-se arquiteto lisboa,2021-10-25,2021-10-25,Opunentes Critérios -LDA,Lisboa,http://www.cargadetrabalhos.net/2021/10/25/pro...
1,"Organização de elevado prestígio, pretende rec...",tutor,2021-10-22,2021-10-25,Anónimo,Margem Sul,http://www.cargadetrabalhos.net/2021/10/22/tut...
2,Referência: PT202110-PRO-SDY,hr project consultant,2021-10-21,2021-10-25,Sendys,Lisboa,http://www.cargadetrabalhos.net/2021/10/21/hr-...
3,"A Astrolabe – Fabricação de Implantes Médicos,...",gestor de qualidade e assuntos regulamentares,2021-10-21,2021-10-25,Astrolabe – Medical,Loures – S.Julião do Tojal,http://www.cargadetrabalhos.net/2021/10/21/ges...
4,"A Erumee, uma Startup Inovadora, especializada...",formadores,2021-10-20,2021-10-25,Erumee,Lisboa,http://www.cargadetrabalhos.net/2021/10/20/rec...


In [63]:
totalJobsByYearMonth(carga_de_trabalhos_clean)

job_title
post_year post_month           
2021      10                102
          9                 108
          8                  30

## Emprego XL

In [64]:
emprego_xl_clean = (emprego_xl.
                    pipe(copy_df).
                    pipe(replacenan).
                    pipe(postDateFillNa).
                    pipe(applyFuncToColumn).
                    pipe(pipeInvertDate).
                    pipe(dropNullJobs).
                    pipe(applyFuncToColumn, function=cleanJobTitle, columns_list=['job_title']).
                    pipe(toDatetime, ['scrape_date', 'post_date']).
                    # # pipe(convertToDatetime, longToShortDate, '/').
                    pipe(removeDupes)
)

print(f'Previous shape: {emprego_xl.shape}\nCurrent shape:{emprego_xl_clean.shape}')
emprego_xl_clean.head()

Previous shape: (39460, 7)
Current shape:(37808, 7)


,job_description,job_title,post_date,scrape_date,company,job_href,job_location
1,Duração do Curso \n1150h \nCertificação \nSIGO...,curso téc. auxiliar de farmácia,2021-10-26,2021-10-26,NaN,https://www.empregoxl.com/emprego/478272/curso...,NaN
2,Duração do Curso \n35h \nCertificação \nSIGO \...,curso de e-formador-tutor,2021-10-26,2021-10-26,NaN,https://www.empregoxl.com/emprego/478270/curso...,NaN
3,Duração do Curso \n125h \nCertificação \nSIGO ...,curso de auxiliar de saúde,2021-10-26,2021-10-26,NaN,https://www.empregoxl.com/emprego/478273/curso...,NaN
4,Duração do Curso \n154h \nCertificação \nSIGO ...,curso de terapeuta spa e wellness,2021-10-26,2021-10-26,NaN,https://www.empregoxl.com/emprego/478274/curso...,NaN
5,Duração do Curso \n150h \nCertificação \nSIGO ...,curso de gestor de formação,2021-10-26,2021-10-26,NaN,https://www.empregoxl.com/emprego/478268/curso...,NaN


In [65]:
totalJobsByYearMonth(emprego_xl_clean)

job_title
post_year post_month           
2021      10               3206
          9                3537
          8                3372
          7                3141
          6                3331
          5                3471
          4                3189
          3                3087
          2                2208
          1                2536
2020      12               2589
          11               4141

## Emprego.Org `SCRAPE AGAIN WITH SCRAPY FOR THE CORRECT FIELDS`

In [66]:
emprego_org.head()

,job_title,job_description,post_date,scrape_date,job_location,job_href,salary,company
0,Consultor(a) Imobiliário(a) – MaisConsultores ...,Gostava de poder mudar a sua vida? De poder te...,2021-10-26 / Viewed 21 times,26/10/2021,Setúbal,https://empregos.org/view.php?job_id=2796901&t...,1250 EUR,None
1,Oferta de emprego: Empregado de Refeitório (M/...,None,2021-10-26 / Viewed 2544 times,26/10/2021,Viana Do Castelo Viana Do Castelo,https://empregos.org/view.php?job_id=2790308&t...,Não especificado,Grupo Trivalor
2,Abertura Nova Filial- Porto/ Gondomar:,Somos uma multinacional de comércio e serviços...,2021-10-25 / Viewed 1969 times,26/10/2021,Porto Porto,https://empregos.org/view.php?job_id=2796890&t...,12000 EUR,None
3,Oferta: Operador Fabril:Grupo Constant,None,2021-10-26 / Viewed 2521 times,26/10/2021,Viana Do Castelo Viana Do Castelo,https://empregos.org/view.php?job_id=2790310&t...,Não especificado,Grupo Constant
4,Oferta de emprego: Empregada de Limpeza (M/F) ...,None,2021-10-26 / Viewed 2505 times,26/10/2021,Viana Do Castelo Viana Do Castelo,https://empregos.org/view.php?job_id=2790312&t...,Não especificado,Eurofirms E.T.T.


In [67]:
emprego_org_clean = (emprego_org.
                    pipe(copy_df).
                    pipe(replacenan).
                    pipe(postDateFillNa).
                    pipe(dropNullJobs).
                    pipe(applyFuncToColumn, function=cleanJobTitle, columns_list=['job_title']).
                    pipe(postDatePreprocess, '/').
                    pipe(toDatetime, ['scrape_date', 'post_date']).
                    pipe(removeDupes)
)

print(f'Previous shape: {emprego_org.shape}\nCurrent shape:{emprego_org_clean.shape}')
emprego_org_clean.head()

Previous shape: (99, 8)
Current shape:(34, 8)


,job_title,job_description,post_date,scrape_date,job_location,job_href,salary,company
0,consultor,Gostava de poder mudar a sua vida? De poder te...,2021-10-26,2021-10-26,Setúbal,https://empregos.org/view.php?job_id=2796901&t...,1250 EUR,None
2,abertura nova filial,Somos uma multinacional de comércio e serviços...,2021-10-25,2021-10-26,Porto Porto,https://empregos.org/view.php?job_id=2796890&t...,12000 EUR,None
22,oferta de emprego: production engineer,Oferta:,2021-10-26,2021-10-26,Viana Do Castelo Viana Do Castelo,https://empregos.org/view.php?job_id=2790328&t...,Não especificado,ENERCON Portugal
24,gestor comercial,A CENTURY 21 Colombo procura pessoas com gosto...,2021-10-26,2021-10-26,LISBOA Lisboa,https://empregos.org/view.php?job_id=2796429&t...,Não especificado,century21
25,oferta: maintenance engineer,Anúncio de emprego:,2021-10-26,2021-10-26,Viana Do Castelo Viana Do Castelo,https://empregos.org/view.php?job_id=2790330&t...,Não especificado,ENERCON Portugal


## ITJOBS

In [68]:
itjobs_clean = (itjobs.
                    pipe(copy_df).
                    pipe(listToRows, 'job_location').
                    pipe(replacenan).
                    pipe(postDateFillNa).
                    pipe(dropNullJobs).
                    pipe(applyFuncToColumn, function=cleanJobTitle, columns_list=['job_title']).
                    pipe(toDatetime, ['scrape_date', 'post_date']).
                    pipe(removeDupes)
)

print(f'Previous shape: {itjobs.shape}\nCurrent shape:{itjobs_clean.shape}')
itjobs_clean.head()

Previous shape: (3890, 9)
Current shape:(5076, 9)


,job_title,job_description,post_date,scrape_date,company,job_location,job_category,job_ref,salary
0,senior ux designer,Job DescriptionOur teams are core to our posit...,2021-10-22 18:30:34,2021-10-22,Dengun,Faro,,https://www.itjobs.pt/oferta/410215/senior-ux-...,NaN
0,senior ux designer,Job DescriptionOur teams are core to our posit...,2021-10-22 18:30:34,2021-10-22,Dengun,Internacional,,https://www.itjobs.pt/oferta/410215/senior-ux-...,NaN
1,técnico de suporte especializado,A Winprovit está a recrutar um Técnico de Supo...,2021-10-22 18:30:31,2021-10-22,Winprovit - Soluções Inteligentes,Lisboa,,https://www.itjobs.pt/oferta/410213/tecnico-de...,NaN
2,senior software engineer,Platforme is a Portuguese technology company f...,2021-10-22 18:30:27,2021-10-22,Platforme,Porto,,https://www.itjobs.pt/oferta/410206/senior-sof...,NaN
3,business support,The world is more connected than ever before a...,2021-10-22 18:30:16,2021-10-22,Axians,Lisboa,,https://www.itjobs.pt/oferta/410199/business-s...,NaN


In [69]:
totalJobsByYearMonth(itjobs_clean)

job_title
post_year post_month           
2021      10               2868
          9                2208

## Jooble

In [70]:
jooble_clean = (jooble.
                    pipe(copy_df).
                    pipe(replacenan).
                    pipe(postDateFillNa).
                    pipe(dropNullJobs).
                    pipe(applyFuncToColumn, function=cleanJobTitle, columns_list=['job_title']).
                    pipe(toDatetime, ['scrape_date', 'post_date']).
                    pipe(removeTags, ['job_title']).
                    pipe(removeDupes)
)

print(f'Previous shape: {jooble.shape}\nCurrent shape:{jooble_clean.shape}')
jooble_clean.head()

Previous shape: (1093, 9)
Current shape:(1093, 9)


,job_title,job_description,post_date,scrape_date,company,job_location,job_category,job_ref,salary
0,empregado de mesa 2ª,Hotel Tivoli Oriente encontra-se a recrutar pa...,2021-10-22,2021-10-22,Hotel Tivoli Oriente,Lisboa,,https://pt.jooble.org/away/-570422328521463595...,
1,recepcionista,Garam Lisboa procura reforçar a sua equipa com...,2021-10-22,2021-10-22,Garam Lisboa,Lisboa,,https://pt.jooble.org/away/2610151826537758894...,
2,assistente de loja,"Organizar a Loja, Receber mercadoria e arruma-...",2021-10-22,2021-10-22,"IS – Recrutamento, Seleção e Formação de Profi...",Porto,,https://pt.jooble.org/away/-561118767370646306...,
3,recepcionista 2ª,O Hotel Tivoli Oriente pretende selecionar um(...,2021-10-22,2021-10-22,Hotel Tivoli Oriente,Lisboa,,https://pt.jooble.org/away/-277454308207513305...,
4,rececionista de 2ª,Funções / Tarefas a Desempenhar:\r\n Atendimen...,2021-10-22,2021-10-22,Hotel Porto Bay Liberdade,Lisboa,,https://pt.jooble.org/away/-158631038483334666...,


In [71]:
totalJobsByYearMonth(jooble_clean)

,,job_title
post_year,post_month,
2021,10,1093


## Landing Jobs IT

In [72]:
landing_jobs_clean = (landing_jobs.
                    pipe(copy_df).
                    pipe(replacenan).
                    pipe(postDateFillNa).
                    pipe(dropNullJobs).
                    pipe(applyFuncToColumn, function=cleanJobTitle, columns_list=['job_title']).
                    pipe(postDatePreprocess, 'T').
                    pipe(toDatetime, ['scrape_date', 'post_date']).
                    # pipe(removeTags, 'job_title').
                    pipe(removeDupes)
)

print(f'Previous shape: {landing_jobs.shape}\nCurrent shape:{landing_jobs_clean.shape}')
landing_jobs_clean.head()

Previous shape: (717, 9)
Current shape:(717, 9)


,job_title,job_description,post_date,scrape_date,company,job_location,job_category,job_ref,salary
0,back-end developer,"We seek flexible people, open to explore diffe...",2018-11-16,2021-10-22,Sky Technology Centre – Portugal,Lisbon,,https://landing.jobs/at/sky-technology-centre-...,
1,mobile developer,Development in languages such as&nbsp;Java or ...,2020-10-29,2021-10-22,Sky Technology Centre – Portugal,Lisbon,,https://landing.jobs/at/sky-technology-centre-...,
2,software engineer,You are a great candidate if you have:Passion ...,2021-06-07,2021-10-22,tb.lx by Daimler Trucks & Buses,Lisbon,,https://landing.jobs/at/tb-lx-by-daimler-truck...,
3,qa engineer,two or more years of professional software tes...,2020-11-13,2021-10-22,Devexperts,Porto,,https://landing.jobs/at/devexperts/qa-engineer...,
4,senior back-end engineer,&nbsp;4+ years experience with Node.js. The Fi...,2021-07-28,2021-10-22,Fidel,Lisbon,,https://landing.jobs/at/fidel/senior-back-end-...,


In [73]:
totalJobsByYearMonth(landing_jobs_clean)

job_title
post_year post_month           
2021      10                  1
          9                  17
          8                   2
          7                   5
          6                  56
          5                 525
          4                  19
          3                  22
          2                  20
          1                   5
2020      12                 16
          11                  4
          10                  8
          9                   1
          8                   4
          7                   3
          6                   1
          5                   2
          4                   1
          3                   4
2018      11                  1

## Net Empregos

In [74]:
net_empregos_clean = (net_empregos.
                    pipe(copy_df).
                    pipe(replacenan).
                    pipe(pipeInvertDate).
                    pipe(postDateFillNa).
                    pipe(dropNullJobs).
                    # two pipes are needed beacause - for some reason, the function was not replacing some words it should
                    pipe(applyFuncToColumn, function=cleanJobTitle, columns_list=['job_title']).
                    pipe(applyFuncToColumn, function=cleanJobTitle, columns_list=['job_title']).
                    pipe(cleanDescription, ['job_title']).
                    pipe(toDatetime, ['scrape_date', 'post_date']).
                    pipe(removeDupes)
)

print(f'Previous shape: {net_empregos.shape}\nCurrent shape:{net_empregos_clean.shape}')
net_empregos_clean.head()

Previous shape: (9012, 8)
Current shape:(9012, 8)


,job_description,job_title,post_date,scrape_date,company,job_location,job_category,job_href
0,Precisa-se Empregada/o de Balcão C/ou S/experi...,empregada de balcão,2021-10-26,2021-10-26,PASTELARIA BUCHINHA,Lisboa,Restauração / Bares / Pastelarias,https://www.net-empregos.com/7817855/precisa-s...
1,O nosso cliente é uma empresa sedeada em Leiri...,gestor técnico comercial,2021-10-26,2021-10-26,"aps - Consultores para Negócios e Gestão, Lda.",Leiria,Engenharia ( Mecanica ),https://www.net-empregos.com/7806454/gestor-te...
2,Função:\rAnálise de projetos e esquemas de cli...,medidor orçamentista,2021-10-26,2021-10-26,Serralharia Cunha,Braga,Engenharia ( Civil ),https://www.net-empregos.com/7816771/medidor-o...
3,O Projeto Face to Face é um método de angariaç...,comercial,2021-10-26,2021-10-26,Fundraisers We Are S.L.,Lisboa,Comercial / Vendas,https://www.net-empregos.com/7815437/comercial...
4,Função:\rCondução de veículo de ligeiros;\rDis...,de motorista de ligeiros,2021-10-26,2021-10-26,Adventure Finder - Lda,Leiria,Transportes / Logística,https://www.net-empregos.com/7816635/precisas-...


In [75]:
totalJobsByYearMonth(net_empregos_clean)

,,job_title
post_year,post_month,
2021,10,9012


# Add Website Column to all dataframes before concat

In [76]:
jobs_dfs = [bons_empregos_clean, career_jet_clean, carga_de_trabalhos_clean, emprego_xl_clean, emprego_org_clean, itjobs_clean, jooble_clean, landing_jobs_clean, net_empregos_clean]
websites = ['Bons empregos', 'Career Jet', 'Carga de Trabalhos', 'Emprego XL', 'Emprego.org','ITjobs','Jooble','Landing Jobs','Net-empregos']

# Add column with website name
for idx, value in enumerate(jobs_dfs):
    value['website'] = websites[idx]

# Concat All dataframes into one for data Deduplication

In [77]:
neworder = ['job_title','job_description','company','job_location','job_category','salary', 'post_date', 'scrape_date','job_href', 'website']

df = pd.concat([i.reindex(columns=neworder) for i in jobs_dfs])

# Validate that the concatenation is happening properly
assert len(df) == sum(len(i) for i in jobs_dfs)

In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66952 entries, 10 to 9011
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   job_title        66952 non-null  object        
 1   job_description  66952 non-null  object        
 2   company          54159 non-null  object        
 3   job_location     62323 non-null  object        
 4   job_category     28870 non-null  object        
 5   salary           12517 non-null  object        
 6   post_date        66952 non-null  datetime64[ns]
 7   scrape_date      66952 non-null  datetime64[ns]
 8   job_href         60066 non-null  object        
 9   website          66952 non-null  object        
dtypes: datetime64[ns](2), object(8)
memory usage: 5.6+ MB


## Pipeline

In [93]:
def cleanCompany(dataframe):
    def capitalize(x):
        try:
            return x.capitalize()
        except AttributeError:
            return ''
    dataframe['company'] = dataframe['company'].apply(lambda x: capitalize(x))
    return dataframe

In [94]:
df_clean = (df.
            pipe(copy_df).
            pipe(replacenan).sort_values(by='post_date').
            pipe(postDateFillNa).
            pipe(dropNullJobs).
            pipe(applyFuncToColumn, function=cleanJobTitle, columns_list=['job_title']).
            pipe(cleanCompany).
            pipe(cleanDescription, ['job_title', 'job_description']).
            pipe(removeDupes, ['job_title', 'company', 'job_location'])
)
df_clean.reset_index(drop=True, inplace=True)

print(f'Previous shape: {df.shape}\nCurrent shape:{df_clean.shape}')

Previous shape: (66952, 10)
Current shape:(66952, 10)


In [95]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66952 entries, 0 to 66951
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   job_title        66952 non-null  object        
 1   job_description  66952 non-null  object        
 2   company          66952 non-null  object        
 3   job_location     62323 non-null  object        
 4   job_category     28870 non-null  object        
 5   salary           12517 non-null  object        
 6   post_date        66952 non-null  datetime64[ns]
 7   scrape_date      66952 non-null  datetime64[ns]
 8   job_href         60066 non-null  object        
 9   website          66952 non-null  object        
dtypes: datetime64[ns](2), object(8)
memory usage: 5.1+ MB


In [96]:
print(f'There is a total of {len(df_clean)} jobs of which {df_clean.job_title.nunique()} have unique titles')

There is a total of 66952 jobs of which 14785 have unique titles


In [97]:
df_clean.describe()

<ipython-input-97-beafcb67837a>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_clean.describe()
<ipython-input-97-beafcb67837a>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df_clean.describe()


,job_title,job_description,company,job_location,job_category,salary,post_date,scrape_date,job_href,website
count,66952,66952,66952,62323,28870,12517,66952,66952,60066,66952
unique,14785,36725,6564,575,68,149,4254,4,50382,9
top,consultor imobiliário,A rede CENTURY 21® é uma das maiores redes imo...,,Lisboa,,,2021-10-22 00:00:00,2021-10-26 00:00:00,http://jobviewtrack.com/pt-pt/job-1d12416e4c16...,Emprego XL
freq,1525,291,20290,27970,17499,6030,10436,46854,40,37808
first,NaN,NaN,NaN,NaN,NaN,NaN,2018-11-16 00:00:00,2021-05-10 00:00:00,NaN,NaN
last,NaN,NaN,NaN,NaN,NaN,NaN,2021-10-26 00:00:00,2021-10-26 00:00:00,NaN,NaN


In [98]:
df_clean.head()

,job_title,job_description,company,job_location,job_category,salary,post_date,scrape_date,job_href,website
0,back-end developer,"We seek flexible people, open to explore diffe...",Sky technology centre – portugal,Lisbon,,,2018-11-16,2021-10-22,NaN,Landing Jobs
1,back-end developer,We hope you haveAt least 3 years working with ...,Blip,Porto,,,2020-03-27,2021-10-22,NaN,Landing Jobs
2,senior back-end developer,We hope you haveAt least 6 years working with ...,Blip,Porto,,,2020-03-27,2021-10-22,NaN,Landing Jobs
3,qa engineer,We hope you haveAt least 3 years working as a ...,Blip,Porto,,,2020-03-27,2021-10-22,NaN,Landing Jobs
4,senior qa engineer,We hope you haveAt least 6 years working as a ...,Blip,Porto,,,2020-03-27,2021-10-22,NaN,Landing Jobs


In [99]:
totalJobsByYearMonth(df_clean)

job_title
post_year post_month           
2021      10              26939
          9                6097
          8                3486
          7                3390
          6                3633
          5                4241
          4                3456
          3                3338
          2                2402
          1                2700
2020      12               2722
          11               4319
          10                212
          9                   1
          8                   4
          7                   3
          6                   1
          5                   2
          4                   1
          3                   4
2018      11                  1

In [557]:
with open(DATA_FOLDER / 'full_data_clean_v1.json', 'w', encoding='utf-8') as file:
    df_clean.to_json(file, force_ascii=False, orient='records')